In [ ]:
import itertools

import numpy as np
import pandas as pd
import altair as alt
import altair_catplot as altcat

import bebi103

import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh.layouts
bokeh.io.output_notebook()
color_palette=['#4e79a7', '#f28e2b', '#e15759', '#76b7b2', '#59a14f', '#edc948', '#b07aa1', '#ff9da7', '#9c755f', '#bab0ac']

In [ ]:
df = pd.read_csv('../data/fret_binding_curve.csv', comment='#')

df

In [ ]:
pri_pred_1 = '''data {
  int N;
  real ca0;
  real cb0[N]; 
  real Kd_sigma;
  real f0_mu;
  real f0_sigma;
  real fq_mu;
  real fq_sigma;
  real noise_sigma;
}

generated quantities{
  real Kd;
  real f0;
  real fq;
  real F[N];
  real temp;
  real noise;
  
  Kd = fabs(normal_rng(0, Kd_sigma));
  f0 = normal_rng(f0_mu, f0_sigma);
  fq = normal_rng(fq_mu, fq_sigma);
  noise = fabs(normal_rng(0, noise_sigma));
  
  
  for (i in 1:N) {
    temp = f0 * ca0 - (2 * (f0 - fq) * ca0 * cb0[i]) / (Kd + ca0 + cb0[i] + sqrt((Kd + ca0 + cb0[i])^2 - 4 * ca0 * cb0[i]));
    F[i] = normal_rng(temp, noise);
  }
}'''

sm = bebi103.stan.StanModel(model_code=pri_pred_1)

In [ ]:
conc_b = df['b conc (nM)'].values

data = dict(N=len(df),
            ca0 = 50,
            cb0 = conc_b,
            Kd_sigma = 150,
            f0_mu = 9000,
            f0_sigma = 2000,
            fq_mu = 4500,
            fq_sigma = 1000,
            noise_sigma = 10000)

samples_gen = sm.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=100)

In [ ]:
df_samples = bebi103.stan.extract_array(samples_gen, name='F')
df_samples.head()

In [ ]:
p = bokeh.plotting.figure(width=750,height=500)

for i in range(100):
    p.line(conc_b[1:], np.diff(df_samples.loc[df_samples['chain_idx'] == i+1, 'F'].values), alpha=0.2, line_width=2)
    
p.line(conc_b[1:], np.diff(df['fluorescence'].values), color='orange',line_width=2)

bokeh.io.show(p)

In [ ]:
p = bokeh.plotting.figure(width=750,height=500)

for i in range(100):
    p.line(conc_b, df_samples.loc[df_samples['chain_idx'] == i+1, 'F'].values, alpha=0.2)
    
p.line(conc_b, df['fluorescence'].values, color='orange',line_width=2)

bokeh.io.show(p)

In [ ]:
def hw92_predictive(df, x, y, namex='index_1', name='F_ppc', perc=[80, 60, 40, 20], 
                    x_axis_label=None, y_axis_label=None, title=None, plot_width=350, plot_height=225, 
                    color='blue', data_color=color_palette[1]):
    '''Mimic of predictive ECDF'''
    
    if color not in ['green', 'blue', 'red', 'gray',
                     'purple', 'orange', 'betancourt']:
        raise RuntimeError("Only allowed colors are 'green', 'blue', 'red', 'gray', 'purple', 'orange'")
    
    colors = {'blue': ['#9ecae1','#6baed6','#4292c6','#2171b5','#084594'],
              'green': ['#a1d99b','#74c476','#41ab5d','#238b45','#005a32'],
              'red': ['#fc9272','#fb6a4a','#ef3b2c','#cb181d','#99000d'],
              'orange': ['#fdae6b','#fd8d3c','#f16913','#d94801','#8c2d04'],
              'purple': ['#bcbddc','#9e9ac8','#807dba','#6a51a3','#4a1486'],
              'gray': ['#bdbdbd','#969696','#737373','#525252','#252525'],
              'betancourt': ['#DCBCBC', '#C79999', '#B97C7C',
                             '#A25050', '#8F2727', '#7C0000']}
    
    p = bokeh.plotting.figure(plot_width=plot_width,
                              plot_height=plot_height,
                              x_axis_label=x_axis_label,
                              y_axis_label=y_axis_label,
                              title=title)
    
    Nb = len(x)
    y_ppc = np.empty((len(perc) * 2 + 1, Nb))
    for i in range(Nb):
        temp = df.loc[df[namex]== i+1, name].values
        y_ppc[-1, i] = np.median(temp)
        for j in range(len(perc)):
            y_ppc[j * 2, i] = np.percentile(temp, 50 - perc[j] / 2)
            y_ppc[j * 2 + 1, i] = np.percentile(temp, 50 + perc[j] / 2)
    
    for j in range(len(perc)):
        bebi103.viz.fill_between(x, y_ppc[j * 2, :],
                     x, y_ppc[j * 2 + 1,:],
                     p=p,
                     show_line=False,
                     fill_color=colors[color][j])
        
    p.line(x, y_ppc[-1, :],
           line_width=2,
           color=colors[color][-1])
    
    p.line(x, y, line_width=2, color='orange')
    
    return p

In [ ]:
p1 = hw92_predictive(df_samples, conc_b, df['fluorescence'].values,name='F', plot_width=500, plot_height=400)

bokeh.io.show(p1)

In [ ]:
model_code_centered = """
data {
  int N;
  int ca0;
  real cb0[N];
  real F[N];
}


parameters {
  real<lower=0> Kd;
  real<lower=0> f0;
  real<lower=0> fq;
  real<lower=0> noise;
}

transformed parameters {
  real F_temp[N];
  for (i in 1:N) {
    F_temp[i] = f0 * ca0 - (2 * (f0 - fq) * ca0 * cb0[i]) / (Kd + ca0 + cb0[i] + sqrt((Kd + ca0 + cb0[i])^2 - 4 * ca0 * cb0[i]));
  }
}


model {
  Kd ~ normal(0, 100);
  f0 ~ normal(9000, 2000);
  fq ~ normal(4500, 1000);
  noise ~ normal(0, 10000);
  
  F ~ normal(F_temp, noise);
}


generated quantities {
  real F_ppc[N];
  
  for (i in 1:N) {
    F_ppc[i] = normal_rng(F_temp[i], noise);
  }
}
"""

sm_centered = bebi103.stan.StanModel(model_code=model_code_centered)

In [ ]:
data = dict(N=len(df),
            ca0 = 50,
            cb0 = conc_b,
            F = df['fluorescence'].values)

In [ ]:
samples_centered = sm_centered.sampling(data=data)
bebi103.stan.check_all_diagnostics(samples_centered)

In [ ]:
bokeh.io.show(bebi103.viz.corner(samples_centered, 
                                 vars=['Kd', 'f0', 'fq','noise'],
                                 plot_width=200,
                                 cmap='gray',
                                 alpha=0.05))

In [ ]:
df_samples_ppc = bebi103.stan.extract_array(samples_centered, name='F_ppc')

In [ ]:
p2 = hw92_predictive(df_samples, conc_b, df['fluorescence'].values,name='F', plot_width=500, plot_height=400)

bokeh.io.show(p2)

In [ ]:
model_code_centered_t = """
data {
  int N;
  int ca0;
  real cb0[N];
  real F[N];
}


parameters {
  real<lower=0> Kd;
  real<lower=0> f0;
  real<lower=0> fq;
  real<lower=1> nu;
  real<lower=0> noise;
}

transformed parameters {
  real F_temp[N];
  for (i in 1:N) {
    F_temp[i] = f0 * ca0 - (2 * (f0 - fq) * ca0 * cb0[i]) / (Kd + ca0 + cb0[i] + sqrt((Kd + ca0 + cb0[i])^2 - 4 * ca0 * cb0[i]));
  }
}


model {
  Kd ~ normal(0, 100);
  f0 ~ normal(9000, 2000);
  fq ~ normal(4500, 1000);
  noise ~ normal(0, 10000);
  nu ~ normal(1,100);
  
  F ~ student_t(nu, F_temp, noise);
}


generated quantities {
  real F_ppc[N];
  
  for (i in 1:N) {
    F_ppc[i] = student_t_rng(nu, F_temp[i], noise);
  }
}
"""

sm_centered_t = bebi103.stan.StanModel(model_code=model_code_centered_t)

In [ ]:
samples_centered_t = sm_centered_t.sampling(data=data)
bebi103.stan.check_all_diagnostics(samples_centered_t)

In [ ]:
bokeh.io.show(bebi103.viz.corner(samples_centered_t, 
                                 vars=['Kd', 'f0', 'fq','noise','nu'],
                                 plot_width=250,
                                 cmap='gray',
                                 alpha=0.05))

In [ ]:
p3 = hw92_predictive(df_samples_ppc, conc_b, df['fluorescence'].values,name='F_ppc', plot_width=500, plot_height=400)

bokeh.io.show(bokeh.layouts.gridplot([[p2, p3]]))

In [ ]:
test = hw92_predictive(df_samples, conc_b, df['fluorescence'].values,name='F',
                       perc=[90, 75, 50, 25], plot_width=500, plot_height=400)

bokeh.io.show(test)

In [ ]:
p = bokeh.plotting.figure(width=750,height=500)

for i in range(4):
    bebi103.viz.fill_between(conc_b, test[2 * i,:], conc_b, test[2 * i + 1, :], show_line=False,p=p)

p.line(conc_b, test[-1, :])
p.line(conc_b, df['fluorescence'].values, color=color_palette[1],line_width=2)

bokeh.io.show(p)

In [ ]:
df_test = pd.DataFrame(np.transpose(test))
df_test['cb0']=conc_b
df_test

In [ ]:
source = bokeh.models.ColumnDataSource?

In [ ]:
p = bokeh.plotting.figure(plot_width=400, plot_height=400)

# add a patch renderer with an alpha an line width
p.patch([1, 2, 3, 4, 5], [6, 7, 8, 7, 3], alpha=0.5, line_width=2)

bokeh.io.show(p)